In [1]:
import os
from cs_sim.batch.batch_corrupt import batch_corrupt_image
from cs_sim.batch.batch_synth import batch_generate_img_with_lines

from torch import optim
from torch.utils.data import DataLoader

from csnet.dataset import CS_Dataset
from csnet.losses import WeightedCrossEntropyLoss, DiceLoss
from csnet.model import CSNet3D

In [2]:
dir_gt = os.path.join('data/gt')
dir_img = os.path.join('data/img')

In [3]:
os.makedirs(dir_gt, exist_ok=True)
os.makedirs(dir_img, exist_ok=True)

batch_generate_img_with_lines(n_img=10, n_jobs=1, dir_out=dir_gt,
                              imgshape=(16, 64, 64), n_lines=10, maxval=255)
batch_corrupt_image(dir_gt, dir_img, n_jobs=1,
                    corruption_steps=[
                        ('poisson_noise', {'snr': 2}),
                        ('convolve', {'sigma': 2}),
                        ('gaussian_noise', {'snr': 50})
                    ])

/research/sharedresources/cbi/common/Anna/anaconda3/envs/quamd/lib/python3.9/site-packages/cs_sim/batch/batch_synth.py:48: UserWarning: data/gt/line_img_00000.tif is a low contrast image
  io.imsave(fn_out, img.astype(__get_type(img)))
/research/sharedresources/cbi/common/Anna/anaconda3/envs/quamd/lib/python3.9/site-packages/cs_sim/batch/batch_synth.py:48: UserWarning: data/gt/line_img_00001.tif is a low contrast image
  io.imsave(fn_out, img.astype(__get_type(img)))
/research/sharedresources/cbi/common/Anna/anaconda3/envs/quamd/lib/python3.9/site-packages/cs_sim/batch/batch_synth.py:48: UserWarning: data/gt/line_img_00002.tif is a low contrast image
  io.imsave(fn_out, img.astype(__get_type(img)))
/research/sharedresources/cbi/common/Anna/anaconda3/envs/quamd/lib/python3.9/site-packages/cs_sim/batch/batch_synth.py:48: UserWarning: data/gt/line_img_00003.tif is a low contrast image
  io.imsave(fn_out, img.astype(__get_type(img)))
/research/sharedresources/cbi/common/Anna/anaconda3/envs

In [4]:
ds = CS_Dataset(dir_img, dir_gt)
dl = DataLoader(ds, batch_size=2, num_workers=2, shuffle=False)
net = CSNet3D(classes=2, channels=1).cuda()
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.0005)

wce_loss = WeightedCrossEntropyLoss().cuda()
dice_loss = DiceLoss().cuda()

/research/sharedresources/cbi/common/Anna/codes/CSNet/csnet/model.py:29: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)


In [5]:
for epoch in range(10):
    print(epoch)
    net.train()
    for idx, batch in enumerate(dl):
        image = batch[0].cuda()
        label = batch[1].cuda()
        optimizer.zero_grad()
        pred = net(image)
        loss = (0.6 * wce_loss(pred, label.squeeze(1)) + 0.4 * dice_loss(pred, label))
        loss.backward()
        optimizer.step()

0


/research/sharedresources/cbi/common/Anna/anaconda3/envs/quamd/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/research/sharedresources/cbi/common/Anna/codes/CSNet/csnet/losses.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  inp = F.softmax(inp)


1
2
3
4
5
6
7
8
9
